In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import random
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pywt import wavedec

In [ ]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          print(os.path.join(dirpath2, f))

In [12]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          model = tf.keras.Sequential([
              # tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              # tf.keras.layers.Dense(32,activation='relu'),

              # tf.keras.layers.Flatten(),
              # tf.keras.layers.Dense(64, activation='relu'),
              # tf.keras.layers.Dense(32, activation='relu'),
              
              # tf.keras.layers.Dense(5, activation='softmax')
              tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              tf.keras.layers.Dense(32,activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),

              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.Dense(32, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              

              tf.keras.layers.Dense(5, activation='softmax')
          ])

          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])

          history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
          # history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

45/45 [==============================] - 2s 40ms/step - loss: 0.9795 - acc: 0.7292 - val_loss: 0.9467 - val_acc: 0.6833
Epoch 5/50
45/45 [==============================] - 2s 39ms/step - loss: 0.8168 - acc: 0.7694 - val_loss: 0.8220 - val_acc: 0.7556
Epoch 6/50
45/45 [==============================] - 2s 35ms/step - loss: 0.6951 - acc: 0.7903 - val_loss: 0.7334 - val_acc: 0.7722
Epoch 7/50
45/45 [==============================] - 2s 37ms/step - loss: 0.6018 - acc: 0.8306 - val_loss: 0.6709 - val_acc: 0.7889
Epoch 8/50
45/45 [==============================] - 2s 34ms/step - loss: 0.5411 - acc: 0.8583 - val_loss: 0.5993 - val_acc: 0.7889
Epoch 9/50
45/45 [==============================] - 1s 29ms/step - loss: 0.4855 - acc: 0.8736 - val_loss: 0.5654 - val_acc: 0.8167
Epoch 10/50
45/45 [==============================] - 1s 33ms/step - loss: 0.4473 - acc: 0.8806 - val_loss: 0.5380 - val_acc: 0.8222
Epoch 11/50
45/45 [==============================] - 1s 29ms/step - loss: 0.4037 - acc: 0.893